# Preprocessing Pipeline

In [2]:
import sys
# sys.path.append("/kaggle/input/python-helpers/")
sys.path.append("../")
import common_utils
from common_utils import (
    check_memory_usage, check_memory_by_global_variable, # For memory handling
    get_time_now, cprint, # For logging
    get_cols, downcast_to_32bit, my_concat, my_power, my_log, list_diff, # For preprocessing
    plot_feature_importance, plot_scatterplot, # For visualization
    get_feature_summary, # For feature engineering & feature selection
    check_auc # For EDA
)
from comp_utils import (
    zero_sum, clean_df
)

In [11]:
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm

In [4]:
%load_ext autoreload
%autoreload

In [5]:
DATA_DIR = '../data/'

## Simple Cleaning
- Remove columns
- Downcast DataType
- Rename columns

In [21]:
%%time
train = pd.read_csv(f"{DATA_DIR}/optiver-trading-at-the-close/train.csv")

CPU times: user 3.68 s, sys: 555 ms, total: 4.24 s
Wall time: 4.24 s


In [22]:
def clean_df(df, columns_to_drop=['row_id', 'time_id']):
    df = df.drop(columns=columns_to_drop, errors="ignore")
    df = downcast_to_32bit(df)
    df = df.rename(
        columns={
            "seconds_in_bucket": "seconds",
            "imbalance_size": "imb_size",
            "imbalance_buy_sell_flag": "imb_flag",
            "reference_price": "ref_price",
            "wap": "wa_price", 
        }
    )
    return df

In [23]:
%%time
train = clean_df(train)
gc.collect()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 188.04it/s]


CPU times: user 161 ms, sys: 209 ms, total: 370 ms
Wall time: 368 ms


90

In [24]:
%%time
train.to_parquet(f'{DATA_DIR}/optiver-train-data/raw_train.parquet')

CPU times: user 1.24 s, sys: 67.7 ms, total: 1.31 s
Wall time: 1.13 s
